Connected to cupy-cuda (Python 3.11.11)

In [ ]:
import scanpy as sc
import os
import configparser
from sklearn.metrics import pairwise_distances

In [ ]:
config = configparser.ConfigParser()
config.read("config.ini")
rawPath = config.get("DEFAULT", "rawPath")
initDir = rawPath + "atlas_annotated/"

adata = sc.read_h5ad(initDir + "atlas_endothelial_filt_norm_nolog.h5ad")

In [ ]:
ori_path = "/group/testa/Project/OvarianAtlas/atlas_project/raw_data/atlas_annotated_backup/atlas_annotated/atlas_endothelial_filt_norm_nolog.h5ad"
adata_ma = sc.read_h5ad(ori_path)

In [ ]:
adata_array = adata.X.toarray()
adata_ma_array = adata_ma.X.toarray()
#correlation_distance = pairwise_distances(adata_array, adata_ma_array, metric="correlation", n_jobs=-1)

# Convert back to correlation similarity if needed
#correlation_similarity = 1 - correlation_distance

In [ ]:
import cupy as cp

def fast_correlation_gpu(expr1, expr2):
    """
    Compute Pearson correlation using GPU with CuPy.
    expr1, expr2: CuPy arrays with shape (N, G), where N is number of cells, G is number of genes.
    """
    expr1 = cp.asarray(expr1)
    expr2 = cp.asarray(expr2)
    # Normalize: zero mean, unit variance for each row (cell)
    expr1 = (expr1 - cp.mean(expr1, axis=1, keepdims=True)) / cp.std(expr1, axis=1, keepdims=True)
    expr2 = (expr2 - cp.mean(expr2, axis=1, keepdims=True)) / cp.std(expr2, axis=1, keepdims=True)

    # Compute correlation using dot product
    corr_matrix = cp.dot(expr1, expr2.T) / expr1.shape[1]
    
    return corr_matrix

# Compute Pearson correlation matrix using CuPy
corr_matrix_gpu = fast_correlation_gpu(adata_array, adata_ma_array)

# Convert to NumPy (if needed for further processing)
corr_matrix_cpu = cp.asnumpy(corr_matrix_gpu)
del corr_matrix_gpu

CompileException: nvrtc: warning: Architectures prior to '<compute/sm>_75' are deprecated and may be removed in a future release
/localscratch/18803265.vittorio.aiello/cupy-cuda/lib/python3.11/site-packages/cupy/_core/include/cupy/carray.cuh(57): catastrophic error: cannot open source file "cuda_fp16.h"
  #include <cuda_fp16.h>
                        ^

1 catastrophic error detected in the compilation of "/tmp/tmpw6xrk103/30959312781f626a575604c913431d79f8b70375.cubin.cu".
Compilation terminated.
